In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [4]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [5]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Lowercasing

In [6]:
df['review'] = df['review'].str.lower()

In [7]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


# Remove HTML tags

In [14]:
text = df['review'][3]
text

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [17]:
import re
def remove_html_tags(text):
    pattern = re.compile("<.*?>")
    return pattern.sub(r'', text)

In [18]:
remove_html_tags(text)

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [19]:
df['review'] = df['review'].apply(remove_html_tags)

In [20]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


# Remove URLS

In [24]:
text1 = 'my name is aniket https://www.kaggle.com/code/aniket62/text-preprocessing'

In [27]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [28]:
remove_url(text1)

'my name is aniket '

# Remove Punctuation

In [30]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
exclude = string.punctuation

In [32]:
def remove_punc(text):
    for char in exclude:
        text = text.replace(char,'')
    return text    

In [33]:
text = 'string. with. Punctuation?'

In [35]:
start = time.time()
print(remove_punc(text))
time1 = time.time() - start
print(time1)

string with Punctuation
0.00016450881958007812


In [36]:
# Another method

def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))

In [37]:
start = time.time()
print(remove_punc1(text))
time2 = time.time() - start
print(time2)

string with Punctuation
0.0003027915954589844


# Chat Word Treatment

In [42]:
shorthand_sentences = {
    "u r gr8": "you are great",
    "pls send me the msg": "please send me the message",
    "idk what u mean": "I don’t know what you mean",
    "omg u did it": "oh my god you did it",
    "brb going 2 eat": "be right back, going to eat",
    "btw congrats on ur bday": "by the way, congratulations on your birthday",
    "tmr we will meet @ 9": "tomorrow we will meet at 9",
    "thx 4 ur help": "thanks for your help",
    "b4 u go call me": "before you go, call me",
    "gn hv a nice sleep": "good night, have a nice sleep",
    "lol that was funny": "laughing out loud, that was funny",
    "cya soon": "see you soon",
    "ttyl bro": "talk to you later, bro",
    "np I got it": "no problem, I got it",
    "gm hv a nice day": "good morning, have a nice day",
    "nvm I figured it out": "never mind, I figured it out",
    "ur work is gr8": "your work is great",
    "wru now": "where are you now",
    "im @ home rn": "I’m at home right now",
    "c u 2mr": "see you tomorrow"
}

In [49]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in shorthand_sentences:
            new_text.append(shorthand_sentences[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)       

In [51]:
chat_conversion("c u 2mr")

'c u 2mr'

# Spelling Correction

In [55]:
#!pip install textblob
from textblob import TextBlob

In [58]:
incorrect_text = "The scientist was fascinateed by the complexity of the universe and spent countless hours studying distant galaxies."

text_blob = TextBlob(incorrect_text)

text_blob.correct().string

'The scientist was fascinate by the complexity of the universe and spent countless hours studying distant salaries.'

# removing Stop Words

In [59]:
from nltk.corpus import stopwords

In [60]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [61]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [62]:
remove_stopwords("This is an example showing how to remove stop words from a sentence.")

'This   example showing   remove stop words   sentence.'

# Handling Emojis

In [63]:
# Remove
import re

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"  # other miscellaneous symbols
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Example usage
text = "I loved the movie. It was 😍🔥👍"
clean_text = remove_emoji(text)
print(clean_text)

I loved the movie. It was 


In [64]:
# Replacing

import emoji
print(emoji.demojize("I loved the movie. It was 😍"))

I loved the movie. It was :smiling_face_with_heart-eyes:


# Tokenization

# (i) Using Split Function 

In [66]:
# Words Tokenization
text = "Hello there! How are you doing today?"
words = text.split()
print(words)

['Hello', 'there!', 'How', 'are', 'you', 'doing', 'today?']


In [65]:
# Sentence Tokenization
paragraph = "Hello there! How are you doing today? I hope you are enjoying NLP."
sentences = paragraph.split(".")
print(sentences)

['Hello there! How are you doing today? I hope you are enjoying NLP', '']


In [67]:
# Problems with split
text = "Hello there! How are you doing today?"
tokens = text.split()
print(tokens)

['Hello', 'there!', 'How', 'are', 'you', 'doing', 'today?']


# (ii) Regular Expression

In [68]:
# Word
import re

text = "Hello there! How are you doing today?"

# Split by non-word characters (anything other than letters, digits, or underscore)
tokens = re.findall(r'\b\w+\b', text)

print(tokens)

['Hello', 'there', 'How', 'are', 'you', 'doing', 'today']


In [69]:
# Sentence
import re

text = "Hello there! How are you doing today? I hope you're learning NLP."

# Split text whenever we see '.', '?', or '!'
sentences = re.split(r'[.!?]', text)

# Remove empty strings
sentences = [s.strip() for s in sentences if s.strip()]

print(sentences)

['Hello there', 'How are you doing today', "I hope you're learning NLP"]


# (iii) NLTK

In [70]:
# Word
from nltk.tokenize import word_tokenize
import nltk

# Download tokenizer models (only once)
nltk.download('punkt')

# Example text
text = "Hello there! How are you doing today?"

# Tokenize into words
words = word_tokenize(text)
print(words)

['Hello', 'there', '!', 'How', 'are', 'you', 'doing', 'today', '?']


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
# Sentence
from nltk.tokenize import sent_tokenize
import nltk

# Download model
nltk.download('punkt')

# Example paragraph
paragraph = "Hello there! How are you doing today? I hope you are enjoying NLP."

# Tokenize into sentences
sentences = sent_tokenize(paragraph)
print(sentences)

['Hello there!', 'How are you doing today?', 'I hope you are enjoying NLP.']


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# (iv) Spacy

In [72]:
# Word
import spacy

nlp = spacy.load("en_core_web_sm")

text = "Hello there! How are you doing today?"

# Process the text
doc = nlp(text)

# Tokenize into words
for token in doc:
    print(token.text)

Hello
there
!
How
are
you
doing
today
?


In [73]:
# Sentence
import spacy

nlp = spacy.load("en_core_web_sm")

text = "Hello there! How are you doing today? I hope you're learning NLP."

doc = nlp(text)

# Sentence tokenization
for sent in doc.sents:
    print(sent.text)

Hello there!
How are you doing today?
I hope you're learning NLP.


# Stemming

In [76]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
sentence = "He was running and eating at the same time. He loves reading books."

words = word_tokenize(sentence)

for w in words:
    print(w, "→", ps.stem(w))


He → he
was → wa
running → run
and → and
eating → eat
at → at
the → the
same → same
time → time
. → .
He → he
loves → love
reading → read
books → book
. → .


# Lemmatization

In [81]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

# Download all required NLTK data files
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')  # ✅ Use this one for new NLTK versions

wordnet_lemmatizer = WordNetLemmatizer()

sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
punctuations = "?:!.,;"

# Tokenize sentence
sentence_words = nltk.word_tokenize(sentence)

# Remove punctuation
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

print("{0:20}{1:20}".format("Word", "Lemma"))

# POS tagging and lemmatization
for word, pos in nltk.pos_tag(sentence_words):
    pos = pos[0].lower()
    pos = pos if pos in ['a', 'r', 'n', 'v'] else None
    if pos:
        print("{0:20}{1:20}".format(word, wordnet_lemmatizer.lemmatize(word, pos='v')))
    else:
        print("{0:20}{1:20}".format(word, wordnet_lemmatizer.lemmatize(word)))


Word                Lemma               
He                  He                  
was                 be                  
running             run                 
and                 and                 
eating              eat                 
at                  at                  
same                same                
time                time                
He                  He                  
has                 have                
bad                 bad                 
habit               habit               
of                  of                  
swimming            swim                
after               after               
playing             play                
long                long                
hours               hours               
in                  in                  
the                 the                 
Sun                 Sun                 


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
